In [ ]:
import pandas as pd

df = pd.read_csv('/content/datasetTeste_Final.csv')

In [ ]:
from re import sub
import nltk
nltk.download('punkt')

def preprocess(text):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    #text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[,!?.\/'+]", " ", text)
    text = sub(r"\+", " mais ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = nltk.word_tokenize(text, language='portuguese')

    #text = text.split()

    return text

In [ ]:
df['cleaned_headlines'] = df['headlines'].apply(lambda x: preprocess(x))

In [ ]:
df

In [ ]:
!wget https://raw.githubusercontent.com/towardsNLP/IMDB-Semantic-Sentiment-Analysis/main/Word2Vec/src/w2v_utils.py

In [ ]:
!pip install -U gensim

In [ ]:
from w2v_utils import w2v_trainer

keyed_vectors, keyed_vocab = w2v_trainer(df['cleaned_headlines'])

In [ ]:
keyed_vectors.most_similar('acusado',topn=50)

In [ ]:
# To make sure that all `positive_concepts` are in the keyed word2vec vocabulary
positive_concepts = ['capaz' , 'abundância' , 'abundante' , 'aclamado' , 'realizar' , 'realizado' , 'realiza' , 'realizando' ,
                     'realização' , 'realizações' , 'conquistar' , 'atingido' , 'conquistas' , 'alcança' , 'alcançar' ,
                     'adequadamente' , 'avanço' , 'avanços' , 'avançando' , 'vantagem' , 'favorecido' , 'vantajoso' ,
                     'vantajosamente' , 'vantagens' , 'aliança' , 'alianças' , 'assegurar' , 'assegurado' , 'assegura' ,
                     'assegurando' , 'atingir' , 'alcançou' , 'atingindo' , 'atinge' , 'atraente' , 'atratividade' , 'bela' ,
                     'belas' , 'benéfico' , 'beneficamente' , 'benefício' , 'beneficiado' , 'beneficiando' , 'melhor' ,
                     'melhor' , 'reforçado' , 'reforçando' , 'reforços' , 'estrondo' , 'crescendo' , 'impulso' , 'impulsionado' ,
                     'brilhante' , 'caridoso' , 'colaborar' , 'colaborou' , 'colabora' , 'colaborando' , 'colaboração' ,
                     'colaborações' , 'colaborativo' , 'colaborador' , 'colaboradores' , 'elogio' , 'cortesia' , 'elogiado' ,
                     'elogiando' , 'elogios' , 'conclusivo' , 'conclusivamente' , 'conducente' , 'confiante' , 'construtivo' ,
                     'construtivamente' , 'cortês' , 'criativo' , 'criativamente' , 'criatividade' , 'deleite' , 'deleitado' ,
                     'delicioso' , 'deliciosamente' , 'deliciando' , 'delícias' , 'confiabilidade' , 'confiável' , 'desejável' ,
                     'desejado' , 'apesar' , 'destinado' , 'diligente' , 'diligentemente' , 'distinção' , 'distinções' ,
                     'distintivo' , 'distintamente' , 'distintividade' , 'sonhe' , 'mais fácil' , 'facilmente' , 'fácil' ,
                     'eficaz' , 'eficiências' , 'eficiência' , 'eficiente' , 'eficientemente' , 'empoderar' , 'empoderado' ,
                     'empoderamento' , 'habilita' , 'habilitar' , 'ativado' , 'permite' , 'possibilitando' , 'encorajado' ,
                     'encorajamento' , 'encorajar' , 'encorajando' , 'realçar' , 'melhorada' , 'aprimoramento' , 'aprimoramentos' ,
                     'aumenta' , 'aprimorando' , 'apreciar' , 'agradável' , 'agradavelmente' , 'gostei' , 'desfrutando' , 'gozo' ,
                     'goza' , 'entusiasmo' , 'entusiasmado' , 'entusiasticamente' , 'excelência' , 'excelente' , 'excede' ,
                     'excepcional' , 'excepcionalmente' , 'animado' , 'excitação' , 'emocionante' , 'exclusivo' , 'exclusivamente' ,
                     'exclusividade' , 'exclusivos' , 'exemplar' , 'fantástico' , 'favorável' , 'favoravelmente' , 'favorecendo' ,
                     'favorito' , 'favoritos' , 'amigáveis' , 'ganho' , 'ganhou' , 'ganhando' , 'ganhos' , 'boa' , 'ótimo' ,
                     'maior' , 'muito' , 'grandeza' , 'feliz' , 'alegremente' , 'felicidade' , 'honra' , 'honroso' , 'honrado' ,
                     'honrando' , 'honras' , 'ideal' , 'impressionar' , 'impressionado' , 'impressiona' , 'impressionando' ,
                     'impressionante' , 'impressionantemente' , 'melhorar' , 'melhorado' , 'melhoria' , 'melhoramentos' ,
                     'melhora' , 'melhorando' , 'incrível' , 'incrivelmente' , 'influente' , 'informativo' , 'ingenuidade' ,
                     'inovar' , 'inovou' , 'inova' , 'inovando' , 'inovação' , 'inovações' , 'inovativa' , 'inovatividade' ,
                     'inovador' , 'inovadores' , 'perspicaz' , 'inspiração' , 'inspirador' , 'integridade' , 'inventar' ,
                     'inventado' , 'inventando' , 'invenção' , 'invenções' , 'inventivo' , 'inventividade' , 'inventor' ,
                     'inventores' , 'liderança' , 'conduzindo' , 'fiel' , 'lucrativo' , 'meritório' , 'oportunidades' ,
                     'oportunidade' , 'otimista' , 'outperform' , 'superou' , 'superando' , 'outperforms' , 'perfeito' ,
                     'aperfeiçoado' , 'perfeitamente' , 'aperfeiçoa' , 'satisfeito' , 'prazer' , 'popular' , 'popularidade' ,
                     'positivo' , 'positivamente' , 'preeminência' , 'preeminente' , 'premier' , 'pré-estreia' , 'prestígio' ,
                     'prestigioso' , 'proativo' , 'proativamente' , 'proficiência' , 'proficiente' , 'proficientemente' ,
                     'rentabilidade' , 'rentável' , 'lucrativamente' , 'progresso' , 'progrediu' , 'progride' , 'progredindo' ,
                     'prosperou' , 'prosperando' , 'prosperidade' , 'próspero' , 'prospera' , 'rebote' , 'recuperou' , 'receptivo' ,
                     'recuperar' , 'recuperando' , 'resolver' , 'revolucionar' , 'revolucionou' , 'revoluciona' , 'revolucionando' ,
                     'recompensa' , 'recompensado' , 'recompensador' , 'recompensas' , 'satisfação' , 'satisfatoriamente' ,
                     'satisfatório' , 'satisfaz' , 'satisfazer' , 'satisfazendo' , 'suave' , 'suavização' , 'suavemente' ,
                     'suaviza' , 'resolve' , 'resolvendo' , 'espetacular' , 'espetacularmente' , 'estabilidade' , 'estabilização' ,
                     'estabilizações' , 'estabilizar' , 'estabilizado' , 'estabiliza' , 'estabilizando' , 'estável' , 'fortalecer' ,
                     'fortalecido' , 'fortalecimento' , 'fortalece' , 'forças' , 'forte' , 'mais forte' , 'suceder' , 'conseguiu' ,
                     'sucedendo' , 'sucede' , 'sucesso' , 'sucessos' , 'bem sucedido' , 'com sucesso' , 'superior' , 'superado' ,
                     'ultrapassa' , 'transparência' , 'tremendo' , 'tremendo' , 'incomparável' , 'inigualável' , 'insuperável' ,
                     'retomada' , 'reviravoltas' , 'valioso' , 'versátil' , 'versatilidade' , 'vibração' , 'vibrante' , 'ganhar' ,
                     'vencedora' , 'vencedores' , 'digno'] 
pos_concepts = [concept for concept in positive_concepts if concept in keyed_vocab]

In [ ]:
negative_concepts = ['abandono' , 'abandonado' , 'abandonando' , 'abandona' , 'abdicado' , 'abdica' , 'abdicar' , 'abdicação' ,
                     'abdicações' , 'aberrante' , 'aberração' , 'aberracional' , 'aberrações' , 'cumplicidade' , 'anormal' ,
                     'anomalias' , 'anormalidade' , 'anormalmente' , 'abolir' , 'abolido' , 'abole' , 'abolição' , 'revogar' ,
                     'anulado' , 'revoga' , 'revogando' , 'revogação' , 'revogações' , 'abrupto' , 'abruptamente' , 'aspereza' ,
                     'ausência' , 'faltas' , 'absentismo' , 'abuso' , 'abusado' , 'abusos' , 'abusando' , 'abusivo' ,
                     'abusivamente' , 'abusividade' , 'acidente' , 'acidental' , 'acidentalmente' , 'acidentes' , 'acusação' ,
                     'acusações' , 'acusar' , 'acusado' , 'acusa' , 'acusando' , 'aquiescer' , 'aquiesceu' , 'consentimentos' ,
                     'aquiescendo' , 'absolver' , 'absolvidos' , 'absolvição' , 'absolvições' , 'absolvido' , 'absolvendo' ,
                     'adulterar' , 'adulterado' , 'adulterando' , 'adulteração' , 'adulterações' , 'adversarial' , 'adversários' ,
                     'adversário' , 'desfavoraveis' , 'adversamente' , 'adversidades' , 'adversidade' , 'rescaldo' , 'rescaldos' ,
                     'contra' , 'agravar' , 'agravado' , 'agrava' , 'agravante' , 'agravamento' , 'agravos' , 'alertado' , 'alerta' ,
                     'alienar' , 'alienado' , 'alienante' , 'alienação' , 'alienações' , 'alegação' , 'alegações' , 'alegar' ,
                     'suposto' , 'alegadamente' , 'alega' , 'alegando' , 'incomodar' , 'aborrecimento' , 'aborrecimentos' , 'irritado' ,
                     'irritante' , 'irrita' , 'anular' , 'anulando' , 'anulação' , 'anulações' , 'anula' , 'anômalo' , 'anomalia' ,
                     'anticoncorrencial' , 'antitruste' , 'discutir' , 'argumentou' , 'argumentando' , 'argumento' , 'argumentativo' ,
                     'argumentos' , 'arrearage' , 'arrearages' , 'atrasos' , 'prender' , 'preso' , 'prisões' , 'artificialmente' ,
                     'assalto' , 'assaltado' , 'assaltos' , 'asserções' , 'atrito' , 'aversamente' , 'backdating' , 'mau' , 'fiança' ,
                     'resgate' , 'balk' , 'empacou' , 'falido' , 'falências' , 'falência' , 'falindo' , 'falidos' , 'proibições' ,
                     'barrado' , 'barreira' , 'barreiras' , 'gargalo' , 'gargalos' , 'boicote' , 'boicotado' , 'boicotes' , 'violação' ,
                     'violado' , 'violações' , 'violar' , 'quebrar' , 'quebra' , 'quebras' , 'demolir' , 'separação' , 'rompe' ,
                     'suborno' , 'subornado' , 'subornos' , 'ponte' , 'quebrado' , 'fardo' , 'sobrecarregado' , 'sobrecarregar' ,
                     'encargos' , 'oneroso' , 'queimado' , 'calamidades' , 'calamitoso' , 'calamidade' , 'cancelar' , 'cancelado' ,
                     'cancelando' , 'cancelamento' , 'cancelamentos' , 'cancela' , 'descuidado' , 'descuidadamente' , 'descuido' ,
                     'catástrofe' , 'catástrofes' , 'catastrófico' , 'catastroficamente' , 'cuidado' , 'cautelar' , 'advertido' ,
                     'alertando' , 'cuidados' , 'cessar' , 'cessou' , 'cessa' , 'cessando' , 'censura' , 'censurado' , 'censuras' ,
                     'censurar' , 'desafio' , 'desafiado' , 'desafios' , 'desafiador' , 'cargas' , 'contornar' , 'contornado' ,
                     'contornando' , 'evasão' , 'desvios' , 'reivindicando' , 'reivindicações' , 'clawback' , 'fechadas' , 'fechar' ,
                     'saldos' , 'fechamento' , 'fechamentos' , 'fecho' , 'coagir' , 'coagido' , 'coage' , 'coerção' , 'coercitivo' ,
                     'colapso' , 'desabou' , 'colapsos' , 'colisão' , 'colisões' , 'conluio' , 'conivente' , 'colusões' , 'colusivo' ,
                     'reclamar' , 'reclamou' , 'reclamando' , 'reclama' , 'queixa' , 'reclamações' , 'complicar' , 'complicado' ,
                     'complica' , 'complicando' , 'complicação' , 'complicações' , 'compulsão' , 'oculto' , 'escondendo' , 'admitir' ,
                     'concedido' , 'admite' , 'concessão' , 'preocupação' , 'preocupado' , 'preocupações' , 'conciliando' , 'conciliação' ,
                     'conciliações' , 'condenar' , 'condenação' , 'condenações' , 'condenado' , 'condena' , 'perdoar' , 'tolerado' ,
                     'confessar' , 'confessado' , 'confessa' , 'confessando' , 'confissão' , 'limitar' , 'confinado' , 'confinamento' ,
                     'confinamentos' , 'confins' , 'confinando' , 'confiscar' , 'confiscado' , 'confisca' , 'confiscando' , 'confisco' ,
                     'confiscos' , 'conflito' , 'em conflito' , 'conflitante' , 'conflitos' , 'enfrentar' , 'confronto' , 'confrontos' ,
                     'confrontado' , 'confrontando' , 'confronta' , 'confundir' , 'confuso' , 'confunde' , 'confusamente' , 'confusão' ,
                     'conspirações' , 'conspiração' , 'conspirador' , 'conspiratório' , 'conspiradores' , 'conspirar' , 'conspirou' ,
                     'conspira' , 'conspirando' , 'desprezo' , 'contender' , 'contendido' , 'contendendo' , 'contende' , 'contenção' ,
                     'contenções' , 'contencioso' , 'contenciosamente' , 'contestado' , 'contestando' , 'contração' , 'contrações' ,
                     'contraditório' , 'contradito' , 'contradizendo' , 'contradição' , 'contradições' , 'contradiz' , 'contrário' ,
                     'controverso' , 'controvérsias' , 'controvérsia' , 'condenando' , 'convicção' , 'convicções' , 'corrigido' ,
                     'corrigindo' , 'correção' , 'correções' , 'corrige' , 'corromper' , 'corrompido' , 'corrompendo' , 'corrupção' ,
                     'corrupções' , 'corruptamente' , 'dispendioso' , 'reconvenção' , 'recontratada' , 'falsificado' , 'falsificador' ,
                     'falsificadores' , 'falsificação' , 'falsificações' , 'contramedida' , 'contramedidas' , 'crime' , 'crimes' ,
                     'criminoso' , 'criminalmente' , 'criminosos' , 'crises' , 'crise' , 'crítico' , 'criticamente' , 'crítica' ,
                     'críticas' , 'criticar' , 'criticado' , 'critica' , 'criticando' , 'crucial' , 'crucialmente' , 'culpabilidade' ,
                     'culpado' , 'culpável' , 'pesado' , 'reduzir' , 'reduzido' , 'reduzindo' , 'contingenciamento' , 'reduções' ,
                     'curtails' , 'cortar' , 'cortes' , 'ataque cibernético' , 'ataques cibernéticos' , 'cyberbullying' , 'cibercrime' ,
                     'crimes cibernéticos' , 'cibercriminoso' , 'cibercriminosos' , 'danificar' , 'estragado' , 'danos' , 'danificando' ,
                     'umedecer' , 'umedecido' , 'perigo' , 'perigoso' , 'perigosamente' , 'perigos' , 'impasse' , 'bloqueado' ,
                     'deadlocking' , 'deadlocks' , 'peso morto' , 'deadweights' , 'exclusão' , 'exclusões' , 'excluído' , 'falecido' ,
                     'engano' , 'enganoso' , 'falsidade' , 'enganar' , 'enganado' , 'engana' , 'enganando' , 'decepção' , 'enganos' ,
                     'enganosamente' , 'declínio' , 'recusado' , 'declínios' , 'declinante' , 'desfigurar' , 'desfigurado' , 'desfiguração' ,
                     'difamação' , 'difamações' , 'difamatório' , 'difamar' , 'difamado' , 'difama' , 'padrão' , 'padronizadas' ,
                     'inadimplente' , 'padrões' , 'derrota' , 'derrotado' , 'derrotar' , 'derrotas' , 'defeito' , 'defeituoso' , 'defeitos' ,
                     'defender' , 'réu' , 'réus' , 'defendido' , 'defendendo' , 'defende' , 'defensiva' , 'adiar' , 'deficiências' ,
                     'deficiência' , 'deficiente' , 'déficit' , 'déficits' , 'defraudar' , 'defraudado' , 'defunto' , 'degradação' ,
                     'degradações' , 'degradar' , 'degradado' , 'degrada' , 'degradante' , 'demora' , 'atrasado' , 'atrasando' ,
                     'deletério' , 'deliberar' , 'deliberado' , 'deliberadamente' , 'inadimplência' , 'delinquência' , 'delinquente' ,
                     'delinqüentemente' , 'delinquentes' , 'delist' , 'deslistagem' , 'delists' , 'morte' , 'demised' , 'demises' ,
                     'demising' , 'demolido' , 'demolição' , 'demolições' , 'rebaixar' , 'rebaixado' , 'rebaixamentos' , 'rebaixando' ,
                     'rebaixamento' , 'negação' , 'negações' , 'negado' , 'nega' , 'denegrir' , 'denegrido' , 'denigre' , 'denegrindo' ,
                     'denigração' , 'negar' , 'negando' , 'esgotar' , 'empobrecido' , 'esgota' , 'esgotando' , 'depleção' , 'depleções' ,
                     'deprecação' , 'abaixar' , 'depressivo' , 'deprime' , 'deprimente' , 'privação' , 'privar' , 'privado' , 'priva' ,
                     'privando' , 'depreciativo' , 'desestabilização' , 'desestabilizar' , 'desestabilizado' , 'desestabilizando' ,
                     'destruir' , 'destruído' , 'destruindo' , 'destrói' , 'destruição' , 'destrutivo' , 'deter' , 'detido' , 'detenção' ,
                     'detenções' , 'dissuadir' , 'deteriorar' , 'deteriorado' , 'se deteriora' , 'deteriorando' , 'deterioração' ,
                     'deteriorações' , 'dissuadido' , 'dissuasão' , 'impedimentos' , 'dissuasivo' , 'deters' , 'diminuir' , 'prejudicou' ,
                     'prejudicando' , 'detrimento' , 'prejudicial' , 'prejudicialmente' , 'detrimentos' , 'desvalorizar' , 'desvalorizado' ,
                     'desvaloriza' , 'desvalorização' , 'devastar' , 'devastado' , 'devastador' , 'devastação' , 'desviar' , 'desviada' ,
                     'desvia' , 'desviando' , 'desvio' , 'devolver' , 'devolvido' , 'deves' , 'devolvendo' , 'difícil' , 'dificuldades' ,
                     'dificilmente' , 'dificuldade' , 'diminuído' , 'diminui' , 'diminuindo' , 'diminuição' , 'desvantagem' ,
                     'desfavorecido' , 'desvantajoso' , 'desvantagens' , 'desfiliação' , 'discordar' , 'desagradável' , 'discordou' ,
                     'discordando' , 'desacordo' , 'desentendimentos' , 'discorda' , 'não permitir' , 'rejeição' , 'nulidades' ,
                     'não permitido' , 'não permitindo' , 'não permite' , 'desaparecer' , 'desaparecimento' , 'desaparecimentos' ,
                     'desaparecido' , 'desaparecendo' , 'desaparece' , 'decepcionar' , 'desapontado' , 'decepcionante' , 'desapontadoramente' ,
                     'desapontamento' , 'decepções' , 'desaponta' , 'desaprovação' , 'reprovações' , 'desaprovar' , 'reprovado' , 'desaprova' ,
                     'desaprovando' , 'dissocia' , 'desassociando' , 'dissociação' , 'dissociações' , 'desastre' , 'desastres' , 'desastroso' ,
                     'desastrosamente' , 'rejeitar' , 'repúdio' , 'disciplinar' , 'renunciado' , 'aviso legal' , 'isenções de responsabilidade' ,
                     'renunciando' , 'divulgar' , 'divulgado' , 'divulga' , 'divulgando' , 'descontinuação' , 'descontinuações' ,
                     'descontinuar' , 'interrompido' , 'interrompe' , 'descontinuando' , 'desencorajar' , 'desencorajado' , 'desencoraja' ,
                     'desencorajando' , 'desacreditar' , 'desacreditado' , 'descrédito' , 'discrepâncias' , 'discrepância' , 'desfavor' ,
                     'desfavorável' , 'desfavorece' , 'vomitar' , 'expelido' , 'desagregação' , 'desagregações' , 'disgorges' , 'despejo' ,
                     'desgraça' , 'vergonhoso' , 'vergonhosamente' , 'desonesto' , 'desonestamente' , 'desonestidade' , 'desonra' , 'desonroso' ,
                     'desonrosamente' , 'desonrado' , 'desincentivos' , 'desinteressado' , 'desinteressadamente' , 'desinteresse' , 'desleal' ,
                     'deslealmente' , 'deslealdade' , 'triste' , 'desanimadamente' , 'dispensar' , 'despedimento' , 'despedimentos' ,
                     'demitido' , 'demite' , 'demitindo' , 'desordenado' , 'depreciar' , 'depreciação' , 'depreciações' , 'depreciativamente' ,
                     'disparidades' , 'disparidade' , 'deslocar' , 'deslocado' , 'deslocamento' , 'deslocamentos' , 'desloca' , 'deslocando' ,
                     'descarte' , 'desapropriar' , 'despossuído' , 'desapropria' , 'desapropriando' , 'desproporção' , 'desproporcional' ,
                     'desproporcionalmente' , 'disputa' , 'disputado' , 'disputas' , 'disputando' , 'desqualificação' , 'desqualificações' ,
                     'desqualificado' , 'desqualifica' , 'desqualificar' , 'desqualificando' , 'desconsiderado' , 'desconsiderando' ,
                     'desconsidera' , 'perturbe' , 'perturbado' , 'atrapalhando' , 'perturbação' , 'interrupções' , 'perturbador' ,
                     'perturba' , 'insatisfação' , 'insatisfeito' , 'dissidência' , 'dissentido' , 'dissidente' , 'dissidentes' ,
                     'discordâncias' , 'dissolução' , 'dissoluções' , 'distorcer' , 'distorcido' , 'distorcendo' , 'distorção' , 'distorções' ,
                     'distorce' , 'distrair' , 'distraído' , 'distração' , 'distrações' , 'distrai' , 'angústia' , 'angustiado' , 'perturbar' ,
                     'distúrbios' , 'desviado' , 'desocupar' , 'desocupado' , 'desinvestimento' , 'desinvestimentos' , 'divórcio' , 'divorciado' ,
                     'divulgação' , 'dúvida' , 'duvidou' , 'duvidoso' , 'dúvidas' , 'redimensionar' , 'downsized' , 'reduções de tamanho' ,
                     'redução de tamanho' , 'downsizings' , 'tempo de inatividade' , 'tempos de inatividade' , 'desaceleração' ,
                     'desacelerações' , 'para baixo' , 'arrasto' , 'drástico' , 'drasticamente' , 'recua' , 'desistiu' , 'seca' , 'secas' ,
                     'coação' , 'disfunção' , 'disfuncional' , 'disfunções' , 'facilitando' , 'notório' , 'notoriamente' , 'embargo' ,
                     'embargado' , 'embargos' , 'embargada' , 'embaraçar' , 'envergonhado' , 'embaraços' , 'embaraçoso' , 'embaraço' ,
                     'constrangimentos' , 'desfalque' , 'embezçador' , 'desfalques' , 'invadir' , 'invadido' , 'invasão' , 'invasões' ,
                     'onerando' , 'encrava' , 'oneração' , 'empecilhos' , 'pôr em perigo' , 'ameaçadas de extinção' , 'colocando em risco' ,
                     'põe em perigo' , 'impor' , 'ordenado' , 'ordenando' , 'corroer' , 'corroído' , 'erode' , 'corroendo' , 'erosão' ,
                     'errático' , 'erraticamente' , 'errou' , 'errando' , 'errôneo' , 'erroneamente' , 'erro' , 'erros' , 'errs' , 'escalar' ,
                     'escalado' , 'escalona' , 'escalando' , 'evadir' , 'evadido' , 'evade' , 'evadindo' , 'evasões' , 'evasivo' , 'expulsar' ,
                     'despejado' , 'evitando' , 'despejos' , 'expulsos' , 'exacerbar' , 'exacerbado' , 'exacerba' , 'exacerbando' ,
                     'exacerbação' , 'exacerbações' , 'exagerar' , 'exagerado' , 'exagera' , 'exagerando' , 'exagero' , 'excessivo' ,
                     'excessivamente' , 'desculpar' , 'exculpado' , 'exclui' , 'desculpando' , 'exculpação' , 'desculpas' , 'desculpatório' ,
                     'exonerar' , 'exonerado' , 'exonera' , 'exonerando' , 'exoneração' , 'exonerações' , 'explorar' , 'exploração' ,
                     'explorações' , 'explorador' , 'explorado' , 'explorando' , 'façanhas' , 'expor' , 'exposto' , 'expõe' , 'expondo' ,
                     'expropriar' , 'expropriado' , 'expropria' , 'expropriando' , 'expropriação' , 'expropriações' , 'expulsão' ,
                     'expulsões' , 'extenuante' , 'falhou' , 'falhando' , 'falhas' , 'falha' , 'fracassos' , 'cair' , 'falso' , 'falsamente' ,
                     'falsifica' , 'falsificar' , 'fatalidades' , 'fatalidade' , 'fatalmente' , 'culpa' , 'com defeito' , 'medo' ,
                     'medos' , 'delitos' , 'criminoso' , 'fictício' , 'multado' , 'multas' , 'disparamos' , 'demissão' , 'falho' , 'proibir' ,
                     'proibido' , 'proibindo' , 'proíbe' , 'força' , 'forçado' , 'forçando' , 'encerrar' , 'encerrado' , 'forecloses' ,
                     'encerramento' , 'execuções hipotecárias' , 'renunciar' , 'antecedentes' , 'perdida' , 'forestall' , 'antecipado' ,
                     'arrasando' , 'forestalls' , 'perdendo' , 'perde' , 'forjadores' , 'fraude' , 'fraudes' , 'fraudulento' , 'fraudulentamente' , 'frívolo' , 'frívola' , 'frustrar' , 'frustrado' , 'frustra' , 'frustrante' , 'frustrantemente' , 'frustração' , 'frustrações' , 'fugitivo' , 'fugitivos' , 'gratuito' , 'gratuitamente' , 'queixas' , 'grosseiramente' , 'infundado' , 'parada' , 'parado' , 'dificultar' , 'dificultado' , 'dificulta' , 'assediar' , 'assediado' , 'assédio' , 'prejuízo' , 'prejudicado' , 'ferozmente' , 'ferir' , 'prejudica' , 'dura' , 'mais duro' , 'mais severo' , 'duramente' , 'arriscado' , 'impedido' , 'obstáculo' , 'obstáculos' , 'hostil' , 'hostilidade' , 'doeu' , 'machucando' , 'ocioso' , 'em ponto morto' , 'em marcha lenta' , 'ignorar' , 'ignorado' , 'ignora' , 'ignorando' , 'eu vou' , 'ilegal' , 'ilegalidades' , 'ilegalidade' , 'ilegalmente' , 'ilegível' , 'ilícito' , 'ilicitamente' , 'ilíquido' , 'iliquidez' , 'desequilíbrio' , 'desequilíbrios' , 'imaturo' , 'imoral' , 'prejudicar' , 'impasses' , 'impedir' , 'impede' , 'impedimento' , 'impedindo' , 'iminente' , 'imperativo' , 'imperfeição' , 'imperfeições' , 'inadmissível' , 'implicar' , 'implicado' , 'implica' , 'implicando' , 'impossibilidade' , 'impossível' , 'aprisionar' , 'aprisionado' , 'impura' , 'impraticável' , 'impraticabilidade' , 'prisão' , 'impróprio' , 'indevidamente' , 'impropriedades' , 'impropriedade' , 'imprudente' , 'imprudentemente' , 'incapacidade' , 'inacessível' , 'imprecisões' , 'imprecisão' , 'impreciso' , 'imprecisamente' , 'inação' , 'inações' , 'inativar' , 'inativado' , 'inactiva' , 'inativando' , 'inativação' , 'inativações' , 'inatividade' , 'insuficiências' , 'inadequação' , 'inadequada' , 'inadequadamente' , 'inadvertido' , 'inadvertidamente' , 'inadvisibilidade' , 'desaconselhável' , 'inapropriado' , 'inapropriadamente' , 'desatenção' , 'incapaz' , 'incapacitado' , 'encarcerar' , 'encarcerado' , 'encarcera' , 'encarcerando' , 'encarceramento' , 'encarceramentos' , 'incidência' , 'incidências' , 'incidente' , 'incidentes' , 'incompatibilidades' , 'incompatibilidade' , 'incompatível' , 'incompetência' , 'incompetente' , 'incompetentemente' , 'incompetentes' , 'incompleto' , 'incompletude' , 'inconclusivo' , 'inconsistências' , 'inconsistência' , 'inconsistente' , 'inconveniência' , 'inconveniências' , 'inconveniente' , 'incorreta' , 'incorretamente' , 'incorreção' , 'indecência' , 'indecente' , 'indefeasible' , 'indefazvelmente' , 'indictable' , 'ineficaz' , 'ineficácia' , 'ineficiências' , 'ineficiência' , 'ineficiente' , 'ineficientemente' , 'inelegibilidade' , 'inelegível' , 'injusto' , 'injustamente' , 'desigualdades' , 'desigualdade' , 'inevitável' , 'inexperiência' , 'inexperiente' , 'inferior' , 'infligido' , 'infração' , 'infrações' , 'infringir' , 'infringido' , 'infracções' , 'infringe' , 'infringindo' , 'inibido' , 'liminar' , 'injunções' , 'ferido' , 'lesões' , 'ferindo' , 'desordenadamente' , 'inquérito' , 'inseguro' , 'insensível' , 'insolvências' , 'insolvência' , 'insolvente' , 'instabilidade' , 'insubordinação' , 'insuficiência' , 'insuficiente' , 'insuficientemente' , 'insurreição' , 'insurreições' , 'intencional' , 'interferir' , 'interferiu' , 'interferência' , 'interferências' , 'interfere' , 'interferindo' , 'intermitente' , 'intermitentemente' , 'interromper' , 'interrompendo' , 'interrupção' , 'intimidação' , 'intrusão' , 'inválido' , 'invalidar' , 'invalidado' , 'invalida' , 'invalidando' , 'invalidação' , 'invalidez' , 'investigar' , 'investigado' , 'investiga' , 'investigando' , 'investigação' , 'investigações' , 'involuntariamente' , 'involuntário' , 'irreconciliável' , 'irreconciliavelmente' , 'irrecuperável' , 'irrecuperavelmente' , 'irregular' , 'irregularidades' , 'irregularidade' , 'irregularmente' , 'irreparável' , 'irreversível' , 'comprometer' , 'ameaçado' , 'justificável' , 'propina' , 'propinas' , 'conscientemente' , 'falta' , 'faltou' , 'em falta' , 'sem brilho' , 'atraso' , 'defasagens' , 'lapso' , 'caducado' , 'lapsos' , 'caducando' , 'tarde' , 'branqueamento' , 'demitir' , 'demissões' , 'mentira' , 'limitação' , 'limitações' , 'persistente' , 'liquidar' , 'liquidado' , 'liquida' , 'liquidando' , 'liquidação' , 'liquidações' , 'liquidante' , 'liquidatários' , 'litigante' , 'litigantes' , 'litigar' , 'litigado' , 'litígios' , 'litigando' , 'litígio' , 'bloqueio' , 'bloqueios' , 'perder' , 'perda' , 'perdas' , 'perdido' , 'deitado' , 'prevaricação' , 'malfeito' , 'avaria' , 'avarias' , 'malícia' , 'malicioso' , 'maliciosamente' , 'negligência' , 'manipular' , 'manipulado' , 'manipula' , 'manipulando' , 'manipulação' , 'manipulações' , 'manipulativo' , 'remarcação' , 'remarcações' , 'aplicação incorreta' , 'erros de aplicação' , 'mal aplicado' , 'aplica erros' , 'aplicar mal' , 'apropriação indevida' , 'apropriação indébita' , 'apropriações indébitas' , 'mal rotulados' , 'calcular mal' , 'calculou mal' , 'calcula mal' , 'calculando mal' , 'erro de cálculo' , 'erros de cálculo' , 'descaracterização' , 'travessura' , 'erro de classificação' , 'erros de classificação' , 'classificado incorretamente' , 'classificar erroneamente' , 'falta de comunicação' , 'má conduta' , 'desatualizado' , 'contravenção' , 'contravenções' , 'mal direcionado' , 'maltratar' , 'maltratado' , 'mishandles' , 'desinformar' , 'desinformação' , 'mal informado' , 'desinformando' , 'informa desinformado' , 'interpretar mal' , 'má interpretação' , 'interpretações erradas' , 'mal interpretado' , 'julgar mal' , 'mal avaliado' , 'julgamentos errados' , 'erro de julgamento' , 'erros de julgamento' , 'errar' , 'rotulado incorretamente' , 'rotulagem incorreta' , 'rótulos errados' , 'induzir em erro' , 'má administração' , 'mal administrado' , 'má gestão' , 'erros de gestão' , 'extraviado' , 'erro de preço' , 'erros de mispricings' , 'deturpar' , 'deturpação' , 'deturpações' , 'deturpado' , 'senhorita' , 'sente falta' , 'erros de conduta' , 'declaração errada' , 'passo em falso' , 'por engano' , 'mistrials' , 'mal entendido' , 'mal entendidos' , 'incompreendido' , 'uso indevido' , 'mal utilizado' , 'monopolista' , 'monopolistas' , 'monopolização' , 'monopolizar' , 'monopolizado' , 'monopoliza' , 'monopolizando' , 'monopólio' , 'moratória' , 'moratórias' , 'na meada' , 'naufrágio' , 'negativo' , 'negativamente' , 'negativos' , 'negligenciado' , 'negligente' , 'negligenciando' , 'negligencia' , 'negligencias' , 'negligentemente' , 'não-entretenimento' , 'não competitivo' , 'não conformidade' , 'não conformidades' , 'não complacente' , 'não complicação' , 'não conforme' , 'não divulgação' , 'não funcional' , 'falta de pagamento' , 'não pagamento' , 'não desempenho' , 'não-desempenho' , 'não produtora' , 'improdutivo' , 'não renovação' , 'incômodo' , 'perturbações' , 'objetado' , 'objeção' , 'censurável' , 'objetivamente' , 'objeções' , 'obsceno' , 'obscenidade' , 'obsolescência' , 'obsoleto' , 'obstruir' , 'obstruído' , 'obstruindo' , 'obstrução' , 'obstruções' , 'ofensa' , 'ofensas' , 'ofender' , 'ofendido' , 'infratores' , 'ofende' , 'omissão' , 'omissões' , 'omitir' , 'omite' , 'omitido' , 'oportunista' , 'oportunisticamente' , 'opor' , 'opôs-se' , 'opõe-se' , 'se opondo' , 'oposição' , 'oposições' , 'falta de energia' , 'fora de moda' , 'excesso de' , 'excedentes' , 'overbuild' , 'excesso de construção' , 'overbuilds' , 'overbuilt' , 'excesso de capacidade' , 'sobrecarga' , 'sobrecargas' , 'superar' , 'supera' , 'superação' , 'superestimar' , 'superestimado' , 'superestima' , 'superestimando' , 'superestimação' , 'superestimações' , 'negligenciar' , 'esquecido' , 'com vista' , 'excesso de pagamento' , 'pagamento indevido' , 'pagamentos indevidos' , 'superproduzido' , 'superprodutos' , 'superprodução' , 'ultrapassagem' , 'superações' , 'ofuscar' , 'ofuscado' , 'ofuscando' , 'obscurece' , 'exageros' , 'excesso de oferta' , 'abertamente' , 'reviravolta' , 'derrubado' , 'derrubando' , 'derruba' , 'supervalorizar' , 'supervalorizado' , 'sobrevalorização' , 'pânico' , 'penalizar' , 'penalizado' , 'penaliza' , 'penalidades' , 'multa' , 'perjúrio' , 'perpetrar' , 'perpetrado' , 'perpetra' , 'perpetrando' , 'perpetração' , 'persistir' , 'persistiu' , 'persistência' , 'persistentemente' , 'persistindo' , 'persiste' , 'penetrante' , 'penetrantemente' , 'penetração' , 'mesquinho' , 'piquete' , 'demandante' , 'demandantes' , 'suplica' , 'suplicou' , 'implorando' , 'articulados' , 'pedidos' , 'pled' , 'pobre' , 'mal' , 'poses' , 'posando' , 'postergar' , 'adiado' , 'adiamento' , 'adiamentos' , 'precipitado' , 'precipitadamente' , 'predatório' , 'preconceito' , 'preconceituoso' , 'preconceitos' , 'prematuro' , 'prematuramente' , 'pressionando' , 'pré-julgamento' , 'prevenção' , 'problema' , 'problemático' , 'problemas' , 'prolongar' , 'prolongamento' , 'prolongamentos' , 'prolongado' , 'prolongando' , 'prolonga' , 'propenso' , 'processar' , 'processado' , 'processos judiciais' , 'protesto' , 'protestado' , 'manifestante' , 'manifestantes' , 'protestando' , 'protestos' , 'protração' , 'provocar' , 'provocado' , 'provoca' , 'provocando' , 'punido' , 'castigar' , 'punir' , 'punição' , 'punições' , 'punitivo' , 'parecer' , 'pretendido' , 'supostamente' , 'pretendendo' , 'pretextos' , 'questão' , 'questionável' , 'questionado' , 'questionando' , 'questões' , 'sair' , 'desistir' , 'trapaceiro' , 'extorsão' , 'racionalização' , 'racionalizações' , 'racionalizar' , 'racionalizado' , 'racionaliza' , 'racionalizando' , 'reavaliação' , 'reavaliações' , 'reatribuir' , 'reatribuído' , 'reatribuição' , 'reatribuições' , 'reatribui' , 'recordar' , 'recordado' , 'recordando' , 'recorda' , 'recessão' , 'recessivo' , 'recessões' , 'imprudente' , 'imprudência' , 'redigir' , 'redigido' , 'redigindo' , 'redação' , 'redações' , 'redefinição' , 'redefinido' , 'redefaults' , 'reparação' , 'repara' , 'reparando' , 'recusa' , 'recusas' , 'recusar' , 'recusou' , 'recusando' , 'rejeitado' , 'rejeitando' , 'rejeições' , 'rejeita' , 'renuncia' , 'renúncia' , 'renúncias' , 'relutância' , 'relutante' , 'renegociar' , 'renegociado' , 'renegociação' , 'renegociações' , 'reparações' ,
                     'recuperado' , 'repossences' , 'repossessing' , 'reintegração de posse' , 'repossessions' , 'repudiar' , 'repudiado' ,
                     'repudia' , 'repudiando' , 'demitir-se' , 'resignado' , 'reafirmar' , 'reafirmado' , 'reformulações' , 'reafirma' ,
                     'reafirmando' , 'reestruturar' , 'reestruturado' , 'reestrutura' , 'reestruturação' , 'reestruturações' , 'retaliar' ,
                     'retaliado' , 'retaliações' , 'retaliação' , 'retribuição' , 'retribuições' , 'revogado' , 'ridículo' , 'ridicularizado' ,
                     'ridículos' , 'ridicularizando' , 'mais arriscado' , 'sabotar' , 'sacrifício' , 'sacrificado' , 'sacrifícios' ,
                     'sacrificial' , 'sacrificando' , 'escandaloso' , 'escândalos' , 'escrutinar' , 'examinado' , 'escrutina' , 'escrutinando' ,
                     'escrutínio' , 'segredo' , 'aproveitar' , 'apreendido' , 'aproveita' , 'apreensão' , 'sério' , 'a sério' , 'gravidade' ,
                     'revés' , 'contratempos' , 'separar' , 'grave' , 'cortado' , 'severamente' , 'severidades' , 'nitidamente' , 'chocado' ,
                     'escassez' , 'encolhimento' , 'encolhimentos' , 'desligar' , 'desligamentos' , 'fecha' , 'fechando' , 'calúnia' ,
                     'caluniado' , 'calunioso' , 'calúnias' , 'deslizamento' , 'derrapagens' , 'lento' , 'desacelere' , 'lentidão' ,
                     'desacelerou' , 'mais devagar' , 'mais lento' , 'desacelerando' , 'lentamente' , 'lerdo' , 'preguiçosamente' ,
                     'solvências' , 'solvência' , 'spam' , 'spammers' , 'spamming' , 'desconcertante' , 'estagnado' , 'estagnar' , 'estagna' ,
                     'estagnando' , 'estagnação' , 'paralisação' , 'paralisações' , 'roubado' , 'parando' , 'pára' , 'tensão' , 'tensas' ,
                     'cepas' , 'estresse' , 'estressado' , 'estressante' , 'estressando' , 'rigoroso' , 'submetido' , 'sujeitando' , 'sujeição' ,
                     'intimação' , 'intimado' , 'intimações' , 'desclassificado' , 'sofra' , 'sofreu' , 'sofrimento' , 'sofre' ,
                     'processando' , 'convocado' , 'convocação' , 'convocações' , 'suscetibilidade' , 'suscetível' , 'suspeito' ,
                     'suspeitos' , 'suspender' , 'suspenso' , 'suspendendo' , 'suspende' , 'suspensão' , 'suspensões' , 'suspeita' ,
                     'suspeitas' , 'desconfiado' , 'mancha' , 'contaminado' , 'contaminação' , 'impurezas' , 'tenso' , 'terminar' ,
                     'terminado' , 'termina' , 'terminando' , 'terminação' , 'terminações' , 'testemunhar' , 'testemunhando' ,
                     'ameaça' , 'ameaçar' , 'ameaçador' , 'ameaças' , 'apertar' , 'tolerar' , 'tolera' , 'tolerando' , 'tolerância' ,
                     'tortuoso' , 'tortuosamente' , 'tragédias' , 'tragédia' , 'trágico' , 'tragicamente' , 'traumático' , 'incomodado' ,
                     'turbulência' , 'inaceitável' , 'inaceitavelmente' , 'não contabilizado' , 'sem aviso prévio' , 'imprevisto' ,
                     'desaprovado' , 'pouco atraente' , 'não autorizado' , 'indisponibilidade' , 'indisponível' , 'inconsciente' ,
                     'incobrável' , 'não recolhido' , 'incobrabilidade' , 'incobráveis' , 'não-competitivo' , 'inconcebível' ,
                     'inconscientemente' , 'incontrolável' , 'incontrolavelmente' , 'descontrolado' , 'não corrigida' , 'descobrir' ,
                     'descoberto' , 'descobrindo' , 'descobre' , 'não-entregável' , 'não entregue' , 'subcapitalizado' , 'rebaixo' ,
                     'ressaltos' , 'subcotação' , 'subestimar' , 'subestimado' , 'subestima' , 'subestimando' , 'subestimação' ,
                     'subfinanciado' , 'underinsured' , 'minar' , 'minado' , 'mina' , 'minando' , 'mal pago' , 'pagamento a menor' ,
                     'pagamentos a menor' , 'underpays' , 'underperform' , 'baixo desempenho' , 'com baixo desempenho' , 'subproduzido' ,
                     'subprodução' , 'subnotificação' , 'eufemismo' , 'subavaliações' , 'subutilização' , 'subutilizado' , 'indesejável' ,
                     'indesejado' , 'não detectado' , 'indeterminado' , 'não revelado' , 'não documentado' , 'indevido' , 'antieconômico' ,
                     'não economicamente' , 'desempregado' , 'desemprego' , 'antiético' , 'sem ética' , 'sem justificativa' , 'inesperado' ,
                     'inesperadamente' , 'injusto' , 'desfavorabilidade' , 'desfavoravelmente' , 'inviável' , 'inapto' , 'inaptidão' ,
                     'imprevisível' , 'infeliz' , 'infelizmente' , 'sem fundamento' , 'sem financiamento' , 'sem seguro' , 'não intencional' ,
                     'injustificável' , 'injustificadamente' , 'injustificado' , 'sem saber' , 'sem licença' , 'sem liquidez' ,
                     'não comercializável' , 'inegociável' , 'não-meritório' , 'desnecessariamente' , 'desnecessário' , 'não remunerado' ,
                     'não realizado' , 'nao planejado' , 'impopular' , 'imprevisibilidade' , 'improfitabilidade' , 'não rentável' ,
                     'não qualificado' , 'irrealista' , 'irracional' , 'irracionalidade' , 'irracionalmente' , 'não receptiva' ,
                     'não recuperado' , 'não reembolsado' , 'não confiável' , 'sem remédio' , 'não declarado' , 'não resolvido' ,
                     'agitação' , 'invendável' , 'insatisfatório' , 'não programado' , 'não vendido' , 'insalubre' , 'instável' ,
                     'mal sucedido' , 'sem sucesso' , 'impropriamente' , 'insuspeito' , 'desavisado' , 'insustentável' , 'intempestivo' ,
                     'inverdade' , 'falsa' , 'sem vergonha' , 'inverdades' , 'inutilizável' , 'sem vontade' , 'falta de vontade' ,
                     'chateado' , 'urgência' , 'urgente' , 'usurário' , 'usurpar' , 'usurpado' , 'usurpação' , 'usurpas' , 'usura' ,
                     'vandalismo' , 'veredito' , 'veredictos' , 'vetado' , 'vítimas' , 'viola' , 'violando' , 'violativo' , 'infrator' ,
                     'violadores' , 'violência' , 'violento' , 'violentamente' , 'viciar' , 'viciado' , 'viciadas' , 'viciando' ,
                     'viciação' , 'volátil' , 'volatilidade' , 'vulnerabilidades' , 'vulnerabilidade' , 'vulnerável' , 'vulneravelmente' ,
                     'advertir' , 'avisou' , 'aviso' , 'advertências' , 'adverte' , 'desperdiçado' , 'desperdício' , 'desperdiçando' ,
                     'fraco' , 'enfraquecer' , 'enfraquecido' , 'enfraquecimento' , 'enfraquece' , 'mais fraco' , 'fracamente' , 'fraqueza' ,
                     'fraquezas' , 'intencionalmente' , 'preocupante' , 'pior' , 'piorar' , 'piorou' , 'piora' , 'inútil' , 'escreva' ,
                     'eliminar' , 'writeoffs' , 'errado' , 'transgressão'] 
neg_concepts = [concept for concept in negative_concepts if concept in keyed_vocab]

In [ ]:
!wget https://raw.githubusercontent.com/towardsNLP/IMDB-Semantic-Sentiment-Analysis/main/Word2Vec/src/w2v_utils.py

In [ ]:
from w2v_utils import overall_semantic_sentiment_analysis, topn_semantic_sentiment_analysis

# Calculating Semantic Sentiment Scores by OSSA model
overall_df_scores = overall_semantic_sentiment_analysis (keyed_vectors = keyed_vectors,
                                                   positive_target_tokens = pos_concepts, 
                                                   negative_target_tokens = neg_concepts,
                                                   doc_tokens = df['cleaned_headlines'])

# Calculating Semantic Sentiment Scores by TopSSA model
topn_df_scores = topn_semantic_sentiment_analysis (keyed_vectors = keyed_vectors,
                                                   positive_target_tokens = pos_concepts, 
                                                   negative_target_tokens = neg_concepts,
                                                   doc_tokens = df['cleaned_headlines'],
                                                     topn=30)

In [ ]:
df['overall_PSS'] = overall_df_scores[0] 
df['overall_NSS'] = overall_df_scores[1] 
df['overall_semantic_sentiment_score'] = overall_df_scores[2] 
df['overall_semantic_sentiment_polarity'] = overall_df_scores[3]

In [ ]:
df['topn_PSS'] = topn_df_scores[0] 
df['topn_NSS'] = topn_df_scores[1] 
df['topn_semantic_sentiment_score'] = topn_df_scores[2] 
df['topn_semantic_sentiment_polarity'] = topn_df_scores[3]

In [ ]:
df['overall_semantic_sentiment_polarity'].value_counts()

In [ ]:
df['topn_semantic_sentiment_polarity'].value_counts()

In [ ]:
df2 = df[['headlines','overall_semantic_sentiment_polarity']]

In [ ]:
df2

In [ ]:
df2 = df2.set_axis(['headlines', 'sentiment'], axis=1, inplace=False)

In [ ]:
df2.to_csv('dataset_FinalVersion.csv')

In [ ]:
df = df[['headlines','overall_semantic_sentiment_score']]
df